In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install python-dotenv
# import kagglehub
# import os
# from dotenv import load_dotenv

# # Zdefiniuj ścieżkę do pliku .env na Twoim Google Drive
# dotenv_path = '/content/drive/MyDrive/API_KEY/kaggle.env'

# # Sprawdź, czy plik istnieje przed próbą jego wczytania
# if os.path.exists(dotenv_path):
#     load_dotenv(dotenv_path)

#     # Pobierz zmienne środowiskowe
#     kaggle_login = os.getenv('KAGGLE_LOGIN')
#     kaggle_key = os.getenv('KAGGLE_KEY')

#     # # Możesz teraz użyć kaggle_login i kaggle_key
#     # print(f"KAGGLE_LOGIN: {kaggle_login}")
#     # print(f"KAGGLE_KEY: {kaggle_key}")
# else:
#     print(f"Plik .env nie został znaleziony w {dotenv_path}")
# os.environ['KAGGLE_USERNAME'] = kaggle_login
# os.environ['KAGGLE_KEY'] = kaggle_key

# kagglehub.login()


In [2]:
# print('Importing data sources...')

# competition_path = kagglehub.competition_download('digit-recognizer')

# print('Data source import complete.')

In [3]:
# # List files in the downloaded directory
# print(f"Files in the downloaded directory ({competition_path}):")
# for dirname, _, filenames in os.walk(competition_path):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# Wczytanie danych
# ds_train = pd.read_csv(os.path.join(competition_path, "train.csv"))
# ds_test = pd.read_csv(os.path.join(competition_path, "test.csv"))
ds_train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
ds_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print(f'Train shape: {ds_train.shape}')
print(f'Test shape: {ds_test.shape}')

2025-07-09 08:10:31.815182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752048632.158108      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752048632.260653      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Train shape: (42000, 785)
Test shape: (28000, 784)


In [5]:
from sklearn.model_selection import train_test_split

X_train_full = ds_train.drop('label', axis=1)
y_train_full = ds_train['label']
y_train = y_train_full

# Normalizacja pikseli

X_train = X_train_full / 255
X_test = ds_test / 255

# # Podział na zbiór treningowy i walidacyjny
# X_train, X_valid = X_train_full[:-2000], X_train_full[-2000:]
# y_train, y_valid = y_train_full[:-2000], y_train_full[-2000:]

In [6]:
# Ustaw ziarno losowości
import random

SEED = 42
# os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [7]:
# Dodanie wymiaru kanału dla "Splotowej Sieci Neuronowej"
X_train = X_train.values.reshape(-1, 28, 28, 1)
# X_valid = X_valid.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)


In [8]:
y_train = y_train.astype('int')

In [9]:
X=X_train
y=y_train

**Stratified Cross Validation**


In [10]:
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping

# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n📂 Fold {fold + 1}")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Model
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.BatchNormalization(),  # dodaj batch norm
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.3),
    
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.3),
    
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    early_stop = EarlyStopping(
        monitor='val_accuracy',
        patience=5,             # zatrzymaj po 5 epokach bez poprawy
        restore_best_weights=True
    )

    # Trening
    history = model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=64,
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=1
    )

    # Ocena
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"✅ Fold {fold+1} Accuracy: {val_acc:.4f}")
    fold_accuracies.append(val_acc)

# Średnia dokładność
mean_acc = np.mean(fold_accuracies)
print(f"\n📊 Średnia dokładność z {skf.n_splits} foldów: {mean_acc:.4f}")


📂 Fold 1


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1752048654.159307      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752048654.159934      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/30


I0000 00:00:1752048659.699141      61 service.cc:148] XLA service 0x7d1b60004e70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752048659.700436      61 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752048659.700457      61 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752048660.187387      61 cuda_dnn.cc:529] Loaded cuDNN version 90300


 27/525 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2624 - loss: 2.8028

...

Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.7337 - loss: 0.9071 - val_accuracy: 0.9699 - val_loss: 0.1135
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9577 - loss: 0.1414 - val_accuracy: 0.9799 - val_loss: 0.0686
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9704 - loss: 0.1005 - val_accuracy: 0.9883 - val_loss: 0.0406
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9769 - loss: 0.0746 - val_accuracy: 0.9848 - val_loss: 0.0501
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9814 - loss: 0.0634 - val_accuracy: 0.9888 - val_loss: 0.0366
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9823 - loss: 0.0601 - val_accuracy: 0.9879 - val_loss: 0.0371
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9848 - loss: 0.0487 - val_accuracy: 0.9899 - val_loss: 0.0331
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9861 - loss: 0.0465 - val_accuracy: 0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7170 - loss: 0.9091 - val_accuracy: 0.9375 - val_loss: 0.2092
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9592 - loss: 0.1383 - val_accuracy: 0.9829 - val_loss: 0.0570
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9714 - loss: 0.0970 - val_accuracy: 0.9861 - val_loss: 0.0449
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9764 - loss: 0.0808 - val_accuracy: 0.9827 - val_loss: 0.0557
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9800 - loss: 0.0670 - val_accuracy: 0.9912 - val_loss: 0.0303
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9843 - loss: 0.0553 - val_accuracy: 0.9895 - val_loss: 0.0355
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9852 - loss: 0.0522 - val_accuracy: 0.9899 - val_loss: 0.0331
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9863 - loss: 0.0459 - val_accuracy: 0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7500 - loss: 0.8171 - val_accuracy: 0.9758 - val_loss: 0.0804
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9562 - loss: 0.1467 - val_accuracy: 0.9860 - val_loss: 0.0478
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9674 - loss: 0.1083 - val_accuracy: 0.9882 - val_loss: 0.0416
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9755 - loss: 0.0825 - val_accuracy: 0.9895 - val_loss: 0.0359
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9779 - loss: 0.0763 - val_accuracy: 0.9881 - val_loss: 0.0400
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9812 - loss: 0.0581 - val_accuracy: 0.9907 - val_loss: 0.0410
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9831 - loss: 0.0538 - val_accuracy: 0.9908 - val_loss: 0.0316
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9857 - loss: 0.0471 - val_accuracy: 0

In [11]:
# history = model.fit(X_train, y_train,
#                     batch_size=64,
#                     epochs=30,
#                     validation_data=(X_valid, y_valid),
#                     shuffle=False,
#                     callbacks=[early_stop])


# def plot_history(history):
#     plt.figure(figsize=(12, 4))

#     # Accuracy
#     plt.subplot(1, 2, 1)
#     plt.plot(history.history['accuracy'], label='train acc')
#     if 'val_accuracy' in history.history:
#         plt.plot(history.history['val_accuracy'], label='val acc')
#     plt.title('Accuracy')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.legend()

#     # Loss
#     plt.subplot(1, 2, 2)
#     plt.plot(history.history['loss'], label='train loss')
#     if 'val_loss' in history.history:
#         plt.plot(history.history['val_loss'], label='val loss')
#     else:
#         print("⚠️ Nie znaleziono 'val_loss' — czy podałeś validation_data?")
#     plt.title('Loss')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

# plot_history(history)

In [12]:
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)

875/875 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [13]:
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(y_pred) + 1),
    "Label": y_pred
})

submission.to_csv("06_submission.csv", index=False)